<a href="https://colab.research.google.com/github/Byeon-MJ/DL_Computer_Vision/blob/main/Face_Landmark_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import cv2
import dlib

In [3]:
# 68개 facial landmark 정의
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))
MOUTH = list(range(48, 68))
NOSE = list(range(48, 68))
EYEBROWS = list(range(17, 27))
JAWLINE = list(range(1, 17))
ALL = list(range(0, 68))

In [4]:
# 원본 동영상 파일
file_name = '/content/gdrive/MyDrive/CV/Face Landmark/myface.mp4'

# output 동영상 이름
output_name = 'myface_output_video.mp4'

# 학습된 dlib 모델
predictor_file = '/content/gdrive/MyDrive/CV/Face Landmark/shape_predictor_68_face_landmarks.dat'

# 얼굴 감지 detector 정의
detector = dlib.get_frontal_face_detector()

# predictor 정의
predictor = dlib.shape_predictor(predictor_file)

In [5]:
def DetectAndDisplay(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 1)   # detector에 의해 얼굴 감지

    for (i, rect) in enumerate(rects):
        # predictor에 의해서 감지된 landmark point를 x, y 좌표로 계산
        points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
        show_parts = points[ALL]

        for (i, point) in enumerate(show_parts):
            x = point[0, 0] # x 좌표 값
            y = point[0, 1] # y 좌표 값 
            cv2.circle(image, (x, y), 1, (0, 255, 255), -1)   # landmark에 점 찍기
            cv2.putText(image, f'{i+1}', (x, y - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1) # 점에 숫자 표시
    
    # video를 disk에 output하기 위해 writer를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                                 (image.shape[1], image.shape[0]), True)
        
    # disk에 frame을 write
    if writer is not None:
        writer.write(image)

In [6]:
# 동영상 읽어옴
cap = cv2.VideoCapture(file_name)

# 원본 동영상이 오픈되는지 확인
writer = None

if not cap.isOpened:
    print('- -(!)Error opening video capture')
    exit(0)

while True:
    # 원본 동영상에서 frame을 읽는다
    ret, image = cap.read()

    # 원본 동영상에서 더 이상 frame을 읽지 못했다면 Exit
    if image is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('- -(!) No captured frame - - Break!')
        break
    
    DetectAndDisplay(image)

- -(!) No captured frame - - Break!
